In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import sys
# import os
# py_file_location = "./drive/My Drive/LAB/COMP90086"
# sys.path.append(os.path.abspath(py_file_location))

# !pip install einops

In [2]:
from environment import *
from utils import *
from model import *

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# ROOT = './drive/My Drive/LAB/COMP90086/COMP90086_2023_TLLdataset 2'
ROOT = './COMP90086_2023_TLLdataset 2'
seed = 19260817
data_list = pd.read_csv(ROOT + '/train.csv')
future_list = pd.read_csv(ROOT + '/test_candidates.csv')

In [4]:
imgs = dict()
for img in tqdm(os.listdir(ROOT+'/train/left')):
  imgs[img] = prepare_image(ROOT+'/train/left/'+img)

for img in tqdm(os.listdir(ROOT+'/train/right')):
  imgs[img] = prepare_image(ROOT+'/train/right/'+img)

for img in tqdm(os.listdir(ROOT+'/test/left')):
  imgs[img] = prepare_image(ROOT+'/test/left/'+img)

for img in tqdm(os.listdir(ROOT+'/test/right')):
  imgs[img] = prepare_image(ROOT+'/test/right/'+img)

100%|██████████| 2000/2000 [00:01<00:00, 1125.06it/s]


In [5]:
train_list, valtest_list = train_test_split(data_list, test_size=0.3, random_state=seed) # train is 70%
val_list, test_list = train_test_split(valtest_list, test_size=0.5, random_state=seed) # val is 15%, test is 15%

In [ ]:
class CNN_Key_Query_Categorisation_CFG:
    num_false               = 1 # how many false samples for each true sample
    target                  = 1 # target loss for correct. false will be 1-target
    encoder                 = 'resnet18' # the name of the model we want to load from torchhub
    freeze_encoder          = False
    input_shape             = (64, 3, 245, 200) # (batch size, 3, [what our loaded in images look like])
    num_mlp_layers          = 1 # how many mlp layers in the end. 0 means no mlp and just 1 out layer; n means (n+1)mlp + out
    hidden_dim              = 512 # hidden dim of mlp
    dropout                 = 0.1 # dropout rate of mlp
    res_learning            = False # whether to use residual layer in mlp
    pretrained              = True # whether to use pretrained encoder
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.BCELoss()   
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving models
    # rootpath                = py_file_location
    DataFactory_Categorisation = DataFactory_Categorisation
    DataLoader_Categorisation = DataLoader_Categorisation
    images                  = imgs

cnn_key_query_categorisation = CNN_Key_Query_Categorisation(CNN_Key_Query_Categorisation_CFG)
cnn_key_query_categorisation.fit(train_list, val_list, batch_size = 64)

In [ ]:
class CNN_Key_Query_Triplet_CFG:
    num_false               = 1 # how many hard negative to try for each anchor
    encoder                 = 'resnet18'
    freeze_encoder          = True
    input_shape             = (64, 3, 245, 200)
    num_mlp_layers          = 1
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 64
    pretrained              = True      
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss() 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs

cnn_key_query_triplet = CNN_Key_Query_Triplet(CNN_Key_Query_Triplet_CFG)
cnn_key_query_triplet.fit(train_list, val_list, batch_size = 64)

# TODO: should we batch normalise...